In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import praw
import prawcore
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import pickle
from random import sample, choice
import concurrent.futures

# List of subreddits to scrape

this list was build from https://anvaka.github.io/redsim. Can be used to expand the list of favourable subreddits.

takeing these for now

In [ ]:
subs = [
    "Sexpolls",
    "sexpositions",
    "Sexconfessional",
    "penissize",
    "masturbation",
    "AskRedditNSFW",
    "sexstories",
    "SexFantasies",
    "sexconfession",
    "askwoman_aboutsex",
    "sextips",
    "sexualhealth",
    "SexPositive",
    "DirtyConfession",
    "Puberty",
    "NSFWIAMA",
    "sexover30",
    "SexToys",
    "sexquestions",
    "deepvaginaproblems",
    "kegels",
    "sexeducation",
    "ColoredLang",
    "masterbationstories",
    "RedditAfterDark",
    "Threesome_advice",
]

# "NSFWIAMA" -> special one (TODO f)

# Scrap these Subreddits

In [ ]:
from utils import scrap_subreddit, init_praw_reddit, save_to_huggingface

In [ ]:
reddit = init_praw_reddit()

# loop through subreddits and save to an intermediate folder
for sub in tqdm(subs):
    try:
        df = scrape_subreddit(sub, reddit)
        if df is not None:
            file_name = f"dataframes/{sub}.csv"
            df.to_csv(file_name, index=False)
            print("subreddit saved to: ", file_name)
    except Exception as e:
        logger.error(f"Error scraping {sub}: {e}")

In [ ]:
import os

# clean the results
files = os.listdir("dataframes/")
dfs = []
for file in files:
    df = pd.read_csv(f"dataframes/{file}")
    dfs.append(df[df["is_question"] & df["is_self"]])

cleaned_df = pd.concat(dfs)
print(cleaned_df.shape, cleaned_df["subreddit"].value_counts())

In [46]:
save_to_huggingface(cleaned_df, name="jjmachan/NSFW-questions-inter-cleaned_df")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jjmachan/.cache/huggingface/datasets/json/default-9433ebf74f6f4bdb/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

# Filter with prosocial

The prosocial.ipynb has the code that converts the `clean_df` intermediate dataset stored in `jjmachan/NSFW-questions-inter-cleaned_df` 

# Get Comments

In [ ]:
get_comments(cleaned_df["post_id"])

In [42]:
nsfw_with_comments = pd.read_csv("df_with_comments.csv").drop_duplicates(subset=["post_id"])
nsfw = pd.read_csv("nsfw.csv").drop_duplicates(subset=["post_id"])
nsfw_final = pd.merge(nsfw.drop(columns=[f"C{i+1}" for i in range(5)]), nsfw_with_comments, on="post_id")
print(nsfw.shape, nsfw_with_comments.shape, nsfw_final.shape)

(12269, 14) (1442, 6) (1442, 14)


In [ ]:
nsfw_final.sample(15)

# Save to Huggingface

In [40]:
from utils import save_to_huggingface

save_to_huggingface(nsfw_final, name="jjmachan/NSFW-questions")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jjmachan/.cache/huggingface/datasets/json/default-424060f196a2282b/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]